# GameTheory 17 - Formalisation Lean : Definitions de Base

**Navigation** : [<< 2-NormalForm (track principal)]([2-NormalForm](GameTheory-2-NormalForm.ipynb)) | [Index](README.md)

**Kernel** : Lean 4 (WSL)

---

## Introduction

Ce notebook inaugure la **Partie 4** de la serie : la **formalisation en Lean 4** des concepts de theorie des jeux. Apres avoir explore les algorithmes et simulations en Python (notebooks 1-16), nous passons maintenant a la **verification formelle**.

### Pourquoi formaliser en Lean ?

| Aspect | Python (simulation) | Lean (formalisation) |
|--------|---------------------|----------------------|
| **Objectif** | Calculer, simuler, visualiser | Prouver, verifier, garantir |
| **Erreurs** | Detectees a l'execution | Impossibles si compile |
| **Theoremes** | Illustres par exemples | Prouves mathematiquement |
| **Confiance** | Tests, intuition | Preuves formelles |

### Isomorphisme de Curry-Howard

Lean repose sur l'**isomorphisme de Curry-Howard** :
- **Propositions = Types** : Une proposition `P` est un type
- **Preuves = Termes** : Une preuve de `P` est un terme de type `P`
- **Implication = Fonction** : Prouver `P → Q` revient a construire une fonction `P → Q`

### Objectifs pedagogiques

1. Definir formellement les structures `Game`, `Strategy`, `Payoff`
2. Formaliser les strategies mixtes via le simplexe standard
3. Definir l'equilibre de Nash et la notion de meilleure reponse
4. Encoder le Dilemme du Prisonnier et verifier ses proprietes

### Prerequis

- Avoir complete les notebooks Python 1-4 (concepts de base)
- Notions de base en Lean 4 (serie SymbolicAI/Lean recommandee)
- Kernel Lean 4 (WSL) installe (voir `scripts/README.md`)

### Duree estimee : 45 minutes

---

## Plan de ce Notebook

1. [Configuration et Imports](#1-configuration)
2. [Structure Game](#2-structure-game)
3. [Strategies Pures et Mixtes](#3-strategies)
4. [Equilibre de Nash](#4-nash-equilibrium)
5. [Exemple : Dilemme du Prisonnier](#5-prisoners-dilemma)
6. [Exercices](#6-exercices)
7. [Solutions](#7-solutions)
8. [Resume](#8-resume)

<a id="1-configuration"></a>
## 1. Configuration

Ce notebook utilise le kernel **Lean 4 (WSL)** qui execute Lean directement dans WSL Ubuntu.

### Configuration requise

Pour utiliser ce notebook :
1. Assurez-vous que le kernel "Lean 4 (WSL)" est installe (voir `install_wsl_kernel.md`)
2. Selectionnez le kernel "Lean 4 (WSL)" dans VSCode

### Alternative : lean_runner.py

Si vous preferez utiliser Python, les notebooks 17 et 18 utilisent `lean_runner.py` qui permet d'executer du code Lean depuis Python.

### Test rapide

Verifions que Lean fonctionne correctement.

In [1]:
#eval 2 + 2
#check Nat

#eval 2 + 2
─────▶  4
#check Nat
──────▶  Nat : Type
--% env 0

Raw input:
{"cmd": "#eval 2 + 2\n#check Nat"}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 1, "column": 0},
   "endPos": {"line": 1, "column": 5},
   "data": "4"},
  {"severity": "info",
   "pos": {"line": 2, "column": 0},
   "endPos": {"line": 2, "column": 6},
   "data": "Nat : Type"}],
 "env": 0}

<a id="2-structure-game"></a>
## 2. Structure Game

### 2.1 Definition minimale

Un **jeu en forme normale** est defini par :
- Un ensemble de **joueurs** `I`
- Pour chaque joueur `i`, un ensemble d'**actions** (ou strategies pures) `A i`
- Pour chaque joueur `i`, une fonction de **gain** qui associe un reel a chaque profil d'actions

En Lean 4, nous pouvons encoder cela avec une structure :

In [2]:
-- Definition de base d'un jeu en forme normale
-- Inspire de math-xmum/Brouwer et MixedMatched/formalizing-game-theory

structure NormalFormGame where
  /-- Ensemble des joueurs -/
  Players : Type
  /-- Ensemble des actions pour chaque joueur -/
  Actions : Players → Type
  /-- Fonction de gain pour chaque joueur -/
  payoff : (i : Players) → ((j : Players) → Actions j) → Int

#check NormalFormGame

-- Definition de base d'un jeu en forme normale
-- Inspire de math-xmum/Brouwer et MixedMatched/formalizing-game-theory

structure NormalFormGame where
  /-- Ensemble des joueurs -/
  Players : Type
  /-- Ensemble des actions pour chaque joueur -/
  Actions : Players → Type
  /-- Fonction de gain pour chaque joueur -/
  payoff : (i : Players) → ((j : Players) → Actions j) → Int

#check NormalFormGame
──────▶  NormalFormGame : Type 1
--% env 1

Raw input:
{"cmd": "-- Definition de base d'un jeu en forme normale\n-- Inspire de math-xmum/Brouwer et MixedMatched/formalizing-game-theory\n\nstructure NormalFormGame where\n  /-- Ensemble des joueurs -/\n  Players : Type\n  /-- Ensemble des actions pour chaque joueur -/\n  Actions : Players \u2192 Type\n  /-- Fonction de gain pour chaque joueur -/\n  payoff : (i : Players) \u2192 ((j : Players) \u2192 Actions j) \u2192 Int\n\n#check NormalFormGame", "env": 0}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 12, "column": 0},
   "endPos": {"line": 12, "column": 6},
   "data": "NormalFormGame : Type 1"}],
 "env": 1}

### 2.2 Jeu fini

Pour les theoremes importants (existence de Nash), nous avons besoin de **jeux finis** ou les ensembles de joueurs et d'actions sont finis.

In [3]:
-- Jeu fini avec contraintes de finitude
structure FiniteGame where
  /-- Nombre de joueurs (utilise Fin n pour avoir exactement n joueurs) -/
  numPlayers : Nat
  /-- Nombre d'actions pour chaque joueur -/
  numActions : Fin numPlayers → Nat
  /-- Fonction de gain : pour chaque joueur, retourne le gain en fonction du profil d'actions -/
  payoff : (i : Fin numPlayers) → ((j : Fin numPlayers) → Fin (numActions j)) → Int

#check FiniteGame

-- Exemple : jeu a 2 joueurs, 2 actions chacun
def twoPlayerTwoActions : FiniteGame := {
  numPlayers := 2
  numActions := fun _ => 2  -- Chaque joueur a 2 actions
  payoff := fun i profile =>
    -- Gains arbitraires pour l'exemple
    if i.val == 0 then
      if profile ⟨0, by omega⟩ == ⟨0, by omega⟩ && profile ⟨1, by omega⟩ == ⟨0, by omega⟩ then 3
      else 0
    else 0
}

#check twoPlayerTwoActions

-- Jeu fini avec contraintes de finitude
structure FiniteGame where
  /-- Nombre de joueurs (utilise Fin n pour avoir exactement n joueurs) -/
  numPlayers : Nat
  /-- Nombre d'actions pour chaque joueur -/
  numActions : Fin numPlayers → Nat
  /-- Fonction de gain : pour chaque joueur, retourne le gain en fonction du profil d'actions -/
  payoff : (i : Fin numPlayers) → ((j : Fin numPlayers) → Fin (numActions j)) → Int

#check FiniteGame
──────▶  FiniteGame : Type

-- Exemple : jeu a 2 joueurs, 2 actions chacun
def twoPlayerTwoActions : FiniteGame := {
  numPlayers := 2
  numActions := fun _ => 2  -- Chaque joueur a 2 actions
  payoff := fun i profile =>
    -- Gains arbitraires pour l'exemple
    if i.val == 0 then
      if profile ⟨0, by omega⟩ == ⟨0, by omega⟩ && profile ⟨1, by omega⟩ == ⟨0, by omega⟩ then 3
      else 0
    else 0
}

#check twoPlayerTwoActions
──────▶  twoPlayerTwoActions : FiniteGame
--% env 2

Raw input:
{"cmd": "-- Jeu fini avec contraintes de finitude\nstructure FiniteGame where\n  /-- Nombre de joueurs (utilise Fin n pour avoir exactement n joueurs) -/\n  numPlayers : Nat\n  /-- Nombre d'actions pour chaque joueur -/\n  numActions : Fin numPlayers \u2192 Nat\n  /-- Fonction de gain : pour chaque joueur, retourne le gain en fonction du profil d'actions -/\n  payoff : (i : Fin numPlayers) \u2192 ((j : Fin numPlayers) \u2192 Fin (numActions j)) \u2192 Int\n\n#check FiniteGame\n\n-- Exemple : jeu a 2 joueurs, 2 actions chacun\ndef twoPlayerTwoActions : FiniteGame := {\n  numPlayers := 2\n  numActions := fun _ => 2  -- Chaque joueur a 2 actions\n  payoff := fun i profile =>\n    -- Gains arbitraires pour l'exemple\n    if i.val == 0 then\n      if profile \u27e80, by omega\u27e9 == \u27e80, by omega\u27e9 && profile \u27e81, by omega\u27e9 == \u27e80, by omega\u27e9 then 3\n      else 0\n    else 0\n}\n\n#check twoPlayerTwoActions", "env": 1}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 10, "column": 0},
   "endPos": {"line": 10, "column": 6},
   "data": "FiniteGame : Type"},
  {"severity": "info",
   "pos": {"line": 24, "column": 0},
   "endPos": {"line": 24, "column": 6},
   "data": "twoPlayerTwoActions : FiniteGame"}],
 "env": 2}

### 2.3 Jeu 2x2 simplifie

Pour les exemples classiques (Prisonnier, Chicken, etc.), un jeu 2x2 est plus pratique :

In [4]:
-- Jeu 2x2 : 2 joueurs, 2 actions chacun
-- Actions : 0 = premiere action, 1 = deuxieme action
structure Game2x2 where
  /-- Matrice des gains du joueur 1 (lignes) -/
  payoff1 : Fin 2 → Fin 2 → Int
  /-- Matrice des gains du joueur 2 (colonnes) -/
  payoff2 : Fin 2 → Fin 2 → Int

-- Helper pour creer un jeu 2x2 a partir de 4 paires de gains
def mkGame2x2 (a11 b11 a12 b12 a21 b21 a22 b22 : Int) : Game2x2 := {
  payoff1 := fun i j =>
    match i.val, j.val with
    | 0, 0 => a11 | 0, 1 => a12
    | 1, 0 => a21 | 1, 1 => a22
    | _, _ => 0  -- Ne devrait pas arriver
  payoff2 := fun i j =>
    match i.val, j.val with
    | 0, 0 => b11 | 0, 1 => b12
    | 1, 0 => b21 | 1, 1 => b22
    | _, _ => 0
}

#check Game2x2
#check mkGame2x2

-- Jeu 2x2 : 2 joueurs, 2 actions chacun
-- Actions : 0 = premiere action, 1 = deuxieme action
structure Game2x2 where
  /-- Matrice des gains du joueur 1 (lignes) -/
  payoff1 : Fin 2 → Fin 2 → Int
  /-- Matrice des gains du joueur 2 (colonnes) -/
  payoff2 : Fin 2 → Fin 2 → Int

-- Helper pour creer un jeu 2x2 a partir de 4 paires de gains
def mkGame2x2 (a11 b11 a12 b12 a21 b21 a22 b22 : Int) : Game2x2 := {
  payoff1 := fun i j =>
    match i.val, j.val with
    | 0, 0 => a11 | 0, 1 => a12
    | 1, 0 => a21 | 1, 1 => a22
    | _, _ => 0  -- Ne devrait pas arriver
  payoff2 := fun i j =>
    match i.val, j.val with
    | 0, 0 => b11 | 0, 1 => b12
    | 1, 0 => b21 | 1, 1 => b22
    | _, _ => 0
}

#check Game2x2
──────▶  Game2x2 : Type
#check mkGame2x2
──────▶  mkGame2x2 (a11 b11 a12 b12 a21 b21 a22 b22 : Int) : Game2x2
--% env 3

Raw input:
{"cmd": "-- Jeu 2x2 : 2 joueurs, 2 actions chacun\n-- Actions : 0 = premiere action, 1 = deuxieme action\nstructure Game2x2 where\n  /-- Matrice des gains du joueur 1 (lignes) -/\n  payoff1 : Fin 2 \u2192 Fin 2 \u2192 Int\n  /-- Matrice des gains du joueur 2 (colonnes) -/\n  payoff2 : Fin 2 \u2192 Fin 2 \u2192 Int\n\n-- Helper pour creer un jeu 2x2 a partir de 4 paires de gains\ndef mkGame2x2 (a11 b11 a12 b12 a21 b21 a22 b22 : Int) : Game2x2 := {\n  payoff1 := fun i j =>\n    match i.val, j.val with\n    | 0, 0 => a11 | 0, 1 => a12\n    | 1, 0 => a21 | 1, 1 => a22\n    | _, _ => 0  -- Ne devrait pas arriver\n  payoff2 := fun i j =>\n    match i.val, j.val with\n    | 0, 0 => b11 | 0, 1 => b12\n    | 1, 0 => b21 | 1, 1 => b22\n    | _, _ => 0\n}\n\n#check Game2x2\n#check mkGame2x2", "env": 2}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 23, "column": 0},
   "endPos": {"line": 23, "column": 6},
   "data": "Game2x2 : Type"},
  {"severity": "info",
   "pos": {"line": 24, "column": 0},
   "endPos": {"line": 24, "column": 6},
   "data": "mkGame2x2 (a11 b11 a12 b12 a21 b21 a22 b22 : Int) : Game2x2"}],
 "env": 3}

<a id="3-strategies"></a>
## 3. Strategies Pures et Mixtes

### 3.1 Strategie pure

Une **strategie pure** est simplement une action choisie par le joueur.

In [5]:
-- Une strategie pure est juste une action
def PureStrategy (g : FiniteGame) (i : Fin g.numPlayers) := Fin (g.numActions i)

-- Un profil de strategies pures : une strategie pour chaque joueur
def PureStrategyProfile (g : FiniteGame) := (i : Fin g.numPlayers) → Fin (g.numActions i)

#check @PureStrategy
#check @PureStrategyProfile

-- Une strategie pure est juste une action
def PureStrategy (g : FiniteGame) (i : Fin g.numPlayers) := Fin (g.numActions i)

-- Un profil de strategies pures : une strategie pour chaque joueur
def PureStrategyProfile (g : FiniteGame) := (i : Fin g.numPlayers) → Fin (g.numActions i)

#check @PureStrategy
──────▶  PureStrategy : (g : FiniteGame) → Fin g.numPlayers → Type
#check @PureStrategyProfile
──────▶  PureStrategyProfile : FiniteGame → Type
--% env 4

Raw input:
{"cmd": "-- Une strategie pure est juste une action\ndef PureStrategy (g : FiniteGame) (i : Fin g.numPlayers) := Fin (g.numActions i)\n\n-- Un profil de strategies pures : une strategie pour chaque joueur\ndef PureStrategyProfile (g : FiniteGame) := (i : Fin g.numPlayers) \u2192 Fin (g.numActions i)\n\n#check @PureStrategy\n#check @PureStrategyProfile", "env": 3}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 7, "column": 0},
   "endPos": {"line": 7, "column": 6},
   "data": "PureStrategy : (g : FiniteGame) → Fin g.numPlayers → Type"},
  {"severity": "info",
   "pos": {"line": 8, "column": 0},
   "endPos": {"line": 8, "column": 6},
   "data": "PureStrategyProfile : FiniteGame → Type"}],
 "env": 4}

### 3.2 Strategie mixte et simplexe standard

Une **strategie mixte** est une distribution de probabilite sur les actions. Mathematiquement, c'est un point du **simplexe standard** :

$$\Delta^{n-1} = \{(p_1, ..., p_n) \in \mathbb{R}^n : p_i \geq 0, \sum_i p_i = 1\}$$

En Lean, nous encodons cela avec un sous-type :

In [6]:
-- Pour les strategies mixtes, nous avons besoin de nombres reels
-- Utilisons Float pour la simplicite (Rat ou Real pour plus de rigueur)

-- Simplexe standard : distribution de probabilite sur n elements
-- C'est un sous-type avec deux conditions :
-- 1. Toutes les probabilites sont >= 0
-- 2. La somme des probabilites = 1

def isNonNeg (f : Fin n → Float) : Prop := ∀ i, f i >= 0

def sumToOne (f : Fin n → Float) : Prop := 
  (List.finRange n).foldl (fun acc i => acc + f i) 0 = 1

-- Le simplexe standard de dimension n-1 (n points)
structure Simplex (n : Nat) where
  prob : Fin n → Float
  nonNeg : ∀ i, prob i >= 0 := by decide
  sumOne : (List.finRange n).foldl (fun acc i => acc + prob i) 0 = 1 := by native_decide

#check Simplex
#check @Simplex.prob

-- Pour les strategies mixtes, nous avons besoin de nombres reels
-- Utilisons Float pour la simplicite (Rat ou Real pour plus de rigueur)

-- Simplexe standard : distribution de probabilite sur n elements
-- C'est un sous-type avec deux conditions :
-- 1. Toutes les probabilites sont >= 0
-- 2. La somme des probabilites = 1

def isNonNeg (f : Fin n → Float) : Prop := ∀ i, f i >= 0

def sumToOne (f : Fin n → Float) : Prop := 
  (List.finRange n).foldl (fun acc i => acc + f i) 0 = 1

-- Le simplexe standard de dimension n-1 (n points)
structure Simplex (n : Nat) where
  prob : Fin n → Float
  nonNeg : ∀ i, prob i >= 0 := by decide
  sumOne : (List.finRange n).foldl (fun acc i => acc + prob i) 0 = 1 := by native_decide

#check Simplex
──────▶  Simplex (n : Nat) : Type
#check @Simplex.prob
──────▶  @Simplex.prob : {n : Nat} → Simplex n → Fin n → Float
--% env 5

Raw input:
{"cmd": "-- Pour les strategies mixtes, nous avons besoin de nombres reels\n-- Utilisons Float pour la simplicite (Rat ou Real pour plus de rigueur)\n\n-- Simplexe standard : distribution de probabilite sur n elements\n-- C'est un sous-type avec deux conditions :\n-- 1. Toutes les probabilites sont >= 0\n-- 2. La somme des probabilites = 1\n\ndef isNonNeg (f : Fin n \u2192 Float) : Prop := \u2200 i, f i >= 0\n\ndef sumToOne (f : Fin n \u2192 Float) : Prop := \n  (List.finRange n).foldl (fun acc i => acc + f i) 0 = 1\n\n-- Le simplexe standard de dimension n-1 (n points)\nstructure Simplex (n : Nat) where\n  prob : Fin n \u2192 Float\n  nonNeg : \u2200 i, prob i >= 0 := by decide\n  sumOne : (List.finRange n).foldl (fun acc i => acc + prob i) 0 = 1 := by native_decide\n\n#check Simplex\n#check @Simplex.prob", "env": 4}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 20, "column": 0},
   "endPos": {"line": 20, "column": 6},
   "data": "Simplex (n : Nat) : Type"},
  {"severity": "info",
   "pos": {"line": 21, "column": 0},
   "endPos": {"line": 21, "column": 6},
   "data": "@Simplex.prob : {n : Nat} → Simplex n → Fin n → Float"}],
 "env": 5}

### 3.3 Strategie mixte pour un jeu

Une strategie mixte pour un joueur dans un jeu fini :

In [7]:
-- Strategie mixte : distribution sur les actions d'un joueur
def MixedStrategy (numActions : Nat) := 
  { f : Fin numActions → Float // (∀ i, f i >= 0) ∧ 
    (List.finRange numActions).foldl (fun acc i => acc + f i) 0 = 1 }

-- Profil de strategies mixtes pour un jeu a 2 joueurs
structure MixedProfile2 (n1 n2 : Nat) where
  sigma1 : Fin n1 → Float  -- Distribution joueur 1
  sigma2 : Fin n2 → Float  -- Distribution joueur 2
  -- Conditions de validite (simplifiees)
  h1_pos : ∀ i, sigma1 i >= 0 := by decide
  h2_pos : ∀ i, sigma2 i >= 0 := by decide

#check @MixedStrategy
#check MixedProfile2

-- Strategie mixte : distribution sur les actions d'un joueur
def MixedStrategy (numActions : Nat) := 
  { f : Fin numActions → Float // (∀ i, f i >= 0) ∧ 
    (List.finRange numActions).foldl (fun acc i => acc + f i) 0 = 1 }

-- Profil de strategies mixtes pour un jeu a 2 joueurs
structure MixedProfile2 (n1 n2 : Nat) where
  sigma1 : Fin n1 → Float  -- Distribution joueur 1
  sigma2 : Fin n2 → Float  -- Distribution joueur 2
  -- Conditions de validite (simplifiees)
  h1_pos : ∀ i, sigma1 i >= 0 := by decide
  h2_pos : ∀ i, sigma2 i >= 0 := by decide

#check @MixedStrategy
──────▶  MixedStrategy : Nat → Type
#check MixedProfile2
──────▶  MixedProfile2 (n1 n2 : Nat) : Type
--% env 6

Raw input:
{"cmd": "-- Strategie mixte : distribution sur les actions d'un joueur\ndef MixedStrategy (numActions : Nat) := \n  { f : Fin numActions \u2192 Float // (\u2200 i, f i >= 0) \u2227 \n    (List.finRange numActions).foldl (fun acc i => acc + f i) 0 = 1 }\n\n-- Profil de strategies mixtes pour un jeu a 2 joueurs\nstructure MixedProfile2 (n1 n2 : Nat) where\n  sigma1 : Fin n1 \u2192 Float  -- Distribution joueur 1\n  sigma2 : Fin n2 \u2192 Float  -- Distribution joueur 2\n  -- Conditions de validite (simplifiees)\n  h1_pos : \u2200 i, sigma1 i >= 0 := by decide\n  h2_pos : \u2200 i, sigma2 i >= 0 := by decide\n\n#check @MixedStrategy\n#check MixedProfile2", "env": 5}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 14, "column": 0},
   "endPos": {"line": 14, "column": 6},
   "data": "MixedStrategy : Nat → Type"},
  {"severity": "info",
   "pos": {"line": 15, "column": 0},
   "endPos": {"line": 15, "column": 6},
   "data": "MixedProfile2 (n1 n2 : Nat) : Type"}],
 "env": 6}

### 3.4 Gain espere

Le gain espere d'un joueur sous un profil de strategies mixtes :

In [8]:
-- Gain espere pour un jeu 2x2 avec strategies mixtes
-- E[u1] = sum_i sum_j sigma1(i) * sigma2(j) * payoff1(i,j)

-- Helper pour convertir Int en Float
def intToFloat (n : Int) : Float := Float.ofInt n

def expectedPayoff1 (g : Game2x2) (s1 : Fin 2 → Float) (s2 : Fin 2 → Float) : Float :=
  let i0 : Fin 2 := ⟨0, by omega⟩
  let i1 : Fin 2 := ⟨1, by omega⟩
  s1 i0 * s2 i0 * intToFloat (g.payoff1 i0 i0) +
  s1 i0 * s2 i1 * intToFloat (g.payoff1 i0 i1) +
  s1 i1 * s2 i0 * intToFloat (g.payoff1 i1 i0) +
  s1 i1 * s2 i1 * intToFloat (g.payoff1 i1 i1)

def expectedPayoff2 (g : Game2x2) (s1 : Fin 2 → Float) (s2 : Fin 2 → Float) : Float :=
  let i0 : Fin 2 := ⟨0, by omega⟩
  let i1 : Fin 2 := ⟨1, by omega⟩
  s1 i0 * s2 i0 * intToFloat (g.payoff2 i0 i0) +
  s1 i0 * s2 i1 * intToFloat (g.payoff2 i0 i1) +
  s1 i1 * s2 i0 * intToFloat (g.payoff2 i1 i0) +
  s1 i1 * s2 i1 * intToFloat (g.payoff2 i1 i1)

#check @expectedPayoff1
#check @expectedPayoff2

-- Gain espere pour un jeu 2x2 avec strategies mixtes
-- E[u1] = sum_i sum_j sigma1(i) * sigma2(j) * payoff1(i,j)

-- Helper pour convertir Int en Float
def intToFloat (n : Int) : Float := Float.ofInt n

def expectedPayoff1 (g : Game2x2) (s1 : Fin 2 → Float) (s2 : Fin 2 → Float) : Float :=
  let i0 : Fin 2 := ⟨0, by omega⟩
  let i1 : Fin 2 := ⟨1, by omega⟩
  s1 i0 * s2 i0 * intToFloat (g.payoff1 i0 i0) +
  s1 i0 * s2 i1 * intToFloat (g.payoff1 i0 i1) +
  s1 i1 * s2 i0 * intToFloat (g.payoff1 i1 i0) +
  s1 i1 * s2 i1 * intToFloat (g.payoff1 i1 i1)

def expectedPayoff2 (g : Game2x2) (s1 : Fin 2 → Float) (s2 : Fin 2 → Float) : Float :=
  let i0 : Fin 2 := ⟨0, by omega⟩
  let i1 : Fin 2 := ⟨1, by omega⟩
  s1 i0 * s2 i0 * intToFloat (g.payoff2 i0 i0) +
  s1 i0 * s2 i1 * intToFloat (g.payoff2 i0 i1) +
  s1 i1 * s2 i0 * intToFloat (g.payoff2 i1 i0) +
  s1 i1 * s2 i1 * intToFloat (g.payoff2 i1 i1)

#check @expectedPayoff1
──────▶  expectedPayoff1 : Game2x2 → (Fin 2 → Float) → (Fin 2 → Float) → Float
#check @expectedPayoff2
──────▶  expectedPayoff2 : Game2x2 → (Fin 2 → Float) → (Fin 2 → Float) → Float
--% env 7

Raw input:
{"cmd": "-- Gain espere pour un jeu 2x2 avec strategies mixtes\n-- E[u1] = sum_i sum_j sigma1(i) * sigma2(j) * payoff1(i,j)\n\n-- Helper pour convertir Int en Float\ndef intToFloat (n : Int) : Float := Float.ofInt n\n\ndef expectedPayoff1 (g : Game2x2) (s1 : Fin 2 \u2192 Float) (s2 : Fin 2 \u2192 Float) : Float :=\n  let i0 : Fin 2 := \u27e80, by omega\u27e9\n  let i1 : Fin 2 := \u27e81, by omega\u27e9\n  s1 i0 * s2 i0 * intToFloat (g.payoff1 i0 i0) +\n  s1 i0 * s2 i1 * intToFloat (g.payoff1 i0 i1) +\n  s1 i1 * s2 i0 * intToFloat (g.payoff1 i1 i0) +\n  s1 i1 * s2 i1 * intToFloat (g.payoff1 i1 i1)\n\ndef expectedPayoff2 (g : Game2x2) (s1 : Fin 2 \u2192 Float) (s2 : Fin 2 \u2192 Float) : Float :=\n  let i0 : Fin 2 := \u27e80, by omega\u27e9\n  let i1 : Fin 2 := \u27e81, by omega\u27e9\n  s1 i0 * s2 i0 * intToFloat (g.payoff2 i0 i0) +\n  s1 i0 * s2 i1 * intToFloat (g.payoff2 i0 i1) +\n  s1 i1 * s2 i0 * intToFloat (g.payoff2 i1 i0) +\n  s1 i1 * s2 i1 * intToFloat (g.payoff2 i1 i1)\n\n#check @expectedPayoff1\n#check @expectedPayoff2", "env": 6}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 23, "column": 0},
   "endPos": {"line": 23, "column": 6},
   "data":
   "expectedPayoff1 : Game2x2 → (Fin 2 → Float) → (Fin 2 → Float) → Float"},
  {"severity": "info",
   "pos": {"line": 24, "column": 0},
   "endPos": {"line": 24, "column": 6},
   "data":
   "expectedPayoff2 : Game2x2 → (Fin 2 → Float) → (Fin 2 → Float) → Float"}],
 "env": 7}

<a id="4-nash-equilibrium"></a>
## 4. Equilibre de Nash

### 4.1 Definition formelle

Un **equilibre de Nash** est un profil de strategies tel qu'aucun joueur ne peut ameliorer son gain en changeant unilateralement de strategie.

Formellement, $(\sigma_1^*, \sigma_2^*)$ est un equilibre de Nash si :
- $\forall \sigma_1, E[u_1(\sigma_1^*, \sigma_2^*)] \geq E[u_1(\sigma_1, \sigma_2^*)]$
- $\forall \sigma_2, E[u_2(\sigma_1^*, \sigma_2^*)] \geq E[u_2(\sigma_1^*, \sigma_2)]$

In [9]:
-- Definition de l'equilibre de Nash pour un jeu 2x2

-- Le joueur 1 joue une meilleure reponse a s2
def isBestResponse1 (g : Game2x2) (s1 : Fin 2 → Float) (s2 : Fin 2 → Float) : Prop :=
  ∀ s1' : Fin 2 → Float, 
    expectedPayoff1 g s1 s2 >= expectedPayoff1 g s1' s2

-- Le joueur 2 joue une meilleure reponse a s1
def isBestResponse2 (g : Game2x2) (s1 : Fin 2 → Float) (s2 : Fin 2 → Float) : Prop :=
  ∀ s2' : Fin 2 → Float,
    expectedPayoff2 g s1 s2 >= expectedPayoff2 g s1 s2'

-- Equilibre de Nash : chaque joueur joue une meilleure reponse
def isNashEquilibrium (g : Game2x2) (s1 : Fin 2 → Float) (s2 : Fin 2 → Float) : Prop :=
  isBestResponse1 g s1 s2 ∧ isBestResponse2 g s1 s2

#check @isNashEquilibrium

-- Definition de l'equilibre de Nash pour un jeu 2x2

-- Le joueur 1 joue une meilleure reponse a s2
def isBestResponse1 (g : Game2x2) (s1 : Fin 2 → Float) (s2 : Fin 2 → Float) : Prop :=
  ∀ s1' : Fin 2 → Float, 
    expectedPayoff1 g s1 s2 >= expectedPayoff1 g s1' s2

-- Le joueur 2 joue une meilleure reponse a s1
def isBestResponse2 (g : Game2x2) (s1 : Fin 2 → Float) (s2 : Fin 2 → Float) : Prop :=
  ∀ s2' : Fin 2 → Float,
    expectedPayoff2 g s1 s2 >= expectedPayoff2 g s1 s2'

-- Equilibre de Nash : chaque joueur joue une meilleure reponse
def isNashEquilibrium (g : Game2x2) (s1 : Fin 2 → Float) (s2 : Fin 2 → Float) : Prop :=
  isBestResponse1 g s1 s2 ∧ isBestResponse2 g s1 s2

#check @isNashEquilibrium
──────▶  isNashEquilibrium : Game2x2 → (Fin 2 → Float) → (Fin 2 → Float) → Prop
--% env 8

Raw input:
{"cmd": "-- Definition de l'equilibre de Nash pour un jeu 2x2\n\n-- Le joueur 1 joue une meilleure reponse a s2\ndef isBestResponse1 (g : Game2x2) (s1 : Fin 2 \u2192 Float) (s2 : Fin 2 \u2192 Float) : Prop :=\n  \u2200 s1' : Fin 2 \u2192 Float, \n    expectedPayoff1 g s1 s2 >= expectedPayoff1 g s1' s2\n\n-- Le joueur 2 joue une meilleure reponse a s1\ndef isBestResponse2 (g : Game2x2) (s1 : Fin 2 \u2192 Float) (s2 : Fin 2 \u2192 Float) : Prop :=\n  \u2200 s2' : Fin 2 \u2192 Float,\n    expectedPayoff2 g s1 s2 >= expectedPayoff2 g s1 s2'\n\n-- Equilibre de Nash : chaque joueur joue une meilleure reponse\ndef isNashEquilibrium (g : Game2x2) (s1 : Fin 2 \u2192 Float) (s2 : Fin 2 \u2192 Float) : Prop :=\n  isBestResponse1 g s1 s2 \u2227 isBestResponse2 g s1 s2\n\n#check @isNashEquilibrium", "env": 7}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 17, "column": 0},
   "endPos": {"line": 17, "column": 6},
   "data":
   "isNashEquilibrium : Game2x2 → (Fin 2 → Float) → (Fin 2 → Float) → Prop"}],
 "env": 8}

### 4.2 Equilibre de Nash en strategies pures

Pour les strategies pures, la definition est plus simple :

In [10]:
-- Equilibre de Nash en strategies pures pour jeu 2x2
def isPureNashEquilibrium (g : Game2x2) (a1 : Fin 2) (a2 : Fin 2) : Prop :=
  -- Joueur 1 ne peut pas ameliorer en changeant d'action
  (∀ a1' : Fin 2, g.payoff1 a1 a2 >= g.payoff1 a1' a2) ∧
  -- Joueur 2 ne peut pas ameliorer en changeant d'action
  (∀ a2' : Fin 2, g.payoff2 a1 a2 >= g.payoff2 a1 a2')

#check @isPureNashEquilibrium

-- Equilibre de Nash en strategies pures pour jeu 2x2
def isPureNashEquilibrium (g : Game2x2) (a1 : Fin 2) (a2 : Fin 2) : Prop :=
  -- Joueur 1 ne peut pas ameliorer en changeant d'action
  (∀ a1' : Fin 2, g.payoff1 a1 a2 >= g.payoff1 a1' a2) ∧
  -- Joueur 2 ne peut pas ameliorer en changeant d'action
  (∀ a2' : Fin 2, g.payoff2 a1 a2 >= g.payoff2 a1 a2')

#check @isPureNashEquilibrium
──────▶  isPureNashEquilibrium : Game2x2 → Fin 2 → Fin 2 → Prop
--% env 9

Raw input:
{"cmd": "-- Equilibre de Nash en strategies pures pour jeu 2x2\ndef isPureNashEquilibrium (g : Game2x2) (a1 : Fin 2) (a2 : Fin 2) : Prop :=\n  -- Joueur 1 ne peut pas ameliorer en changeant d'action\n  (\u2200 a1' : Fin 2, g.payoff1 a1 a2 >= g.payoff1 a1' a2) \u2227\n  -- Joueur 2 ne peut pas ameliorer en changeant d'action\n  (\u2200 a2' : Fin 2, g.payoff2 a1 a2 >= g.payoff2 a1 a2')\n\n#check @isPureNashEquilibrium", "env": 8}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 8, "column": 0},
   "endPos": {"line": 8, "column": 6},
   "data": "isPureNashEquilibrium : Game2x2 → Fin 2 → Fin 2 → Prop"}],
 "env": 9}

### 4.3 Proprietes de base

Quelques proprietes simples des equilibres de Nash :

In [11]:
-- Propriete : un equilibre en strategies pures est aussi un equilibre en strategies mixtes
-- (quand on identifie une strategie pure avec la distribution degeneree)

-- Strategie pure comme strategie mixte degeneree
def pureToMixed (a : Fin 2) : Fin 2 → Float :=
  fun i => if i == a then 1.0 else 0.0

#check @pureToMixed

-- Theoreme (enonce) : si (a1, a2) est un equilibre de Nash pur,
-- alors (pureToMixed a1, pureToMixed a2) est un equilibre de Nash mixte
-- La preuve complete necessite plus de travail sur les Float
theorem pure_nash_implies_mixed_nash (g : Game2x2) (a1 a2 : Fin 2)
    (h : isPureNashEquilibrium g a1 a2) :
    True := by  -- Simplifie pour l'exemple
  trivial

-- Propriete : un equilibre en strategies pures est aussi un equilibre en strategies mixtes
-- (quand on identifie une strategie pure avec la distribution degeneree)

-- Strategie pure comme strategie mixte degeneree
def pureToMixed (a : Fin 2) : Fin 2 → Float :=
  fun i => if i == a then 1.0 else 0.0

#check @pureToMixed
──────▶  pureToMixed : Fin 2 → Fin 2 → Float

-- Theoreme (enonce) : si (a1, a2) est un equilibre de Nash pur,
-- alors (pureToMixed a1, pureToMixed a2) est un equilibre de Nash mixte
-- La preuve complete necessite plus de travail sur les Float
theorem pure_nash_implies_mixed_nash (g : Game2x2) (a1 a2 : Fin 2)
    (h : isPureNashEquilibrium g a1 a2) :
     ─▶ 🟨 unused variable `h`

Note: This linter can be disabled with `set_option linter.unusedVariables false`
    True := by  -- Simplifie pour l'exemple
  trivial
--% env 10

Raw input:
{"cmd": "-- Propriete : un equilibre en strategies pures est aussi un equilibre en strategies mixtes\n-- (quand on identifie une strategie pure avec la distribution degeneree)\n\n-- Strategie pure comme strategie mixte degeneree\ndef pureToMixed (a : Fin 2) : Fin 2 \u2192 Float :=\n  fun i => if i == a then 1.0 else 0.0\n\n#check @pureToMixed\n\n-- Theoreme (enonce) : si (a1, a2) est un equilibre de Nash pur,\n-- alors (pureToMixed a1, pureToMixed a2) est un equilibre de Nash mixte\n-- La preuve complete necessite plus de travail sur les Float\ntheorem pure_nash_implies_mixed_nash (g : Game2x2) (a1 a2 : Fin 2)\n    (h : isPureNashEquilibrium g a1 a2) :\n    True := by  -- Simplifie pour l'exemple\n  trivial", "env": 9}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 8, "column": 0},
   "endPos": {"line": 8, "column": 6},
   "data": "pureToMixed : Fin 2 → Fin 2 → Float"},
  {"severity": "warning",
   "pos": {"line": 14, "column": 5},
   "endPos": {"line": 14, "column": 6},
   "data":
   "unused variable `h`\n\nNote: This linter can be disabled with `set_option linter.unusedVariables false`"}],
 "env": 10}

<a id="5-prisoners-dilemma"></a>
## 5. Exemple : Dilemme du Prisonnier

### 5.1 Definition du jeu

Le **Dilemme du Prisonnier** classique :

|  | Cooperer (0) | Trahir (1) |
|--|--------------|------------|
| **Cooperer (0)** | (3, 3) | (0, 5) |
| **Trahir (1)** | (5, 0) | (1, 1) |

In [12]:
-- Dilemme du Prisonnier
-- Actions : 0 = Cooperer, 1 = Trahir
-- Gains : (C,C)=(3,3), (C,T)=(0,5), (T,C)=(5,0), (T,T)=(1,1)

def prisonersDilemma : Game2x2 := {
  payoff1 := fun i j =>
    match i.val, j.val with
    | 0, 0 => 3  -- (C, C)
    | 0, 1 => 0  -- (C, T)
    | 1, 0 => 5  -- (T, C)
    | 1, 1 => 1  -- (T, T)
    | _, _ => 0
  payoff2 := fun i j =>
    match i.val, j.val with
    | 0, 0 => 3  -- (C, C)
    | 0, 1 => 5  -- (C, T) - Joueur 2 trahit
    | 1, 0 => 0  -- (T, C)
    | 1, 1 => 1  -- (T, T)
    | _, _ => 0
}

#check prisonersDilemma

-- Verification des gains
#eval prisonersDilemma.payoff1 ⟨0, by omega⟩ ⟨0, by omega⟩  -- C,C -> 3
#eval prisonersDilemma.payoff1 ⟨1, by omega⟩ ⟨0, by omega⟩  -- T,C -> 5
#eval prisonersDilemma.payoff1 ⟨1, by omega⟩ ⟨1, by omega⟩  -- T,T -> 1

-- Dilemme du Prisonnier
-- Actions : 0 = Cooperer, 1 = Trahir
-- Gains : (C,C)=(3,3), (C,T)=(0,5), (T,C)=(5,0), (T,T)=(1,1)

def prisonersDilemma : Game2x2 := {
  payoff1 := fun i j =>
    match i.val, j.val with
    | 0, 0 => 3  -- (C, C)
    | 0, 1 => 0  -- (C, T)
    | 1, 0 => 5  -- (T, C)
    | 1, 1 => 1  -- (T, T)
    | _, _ => 0
  payoff2 := fun i j =>
    match i.val, j.val with
    | 0, 0 => 3  -- (C, C)
    | 0, 1 => 5  -- (C, T) - Joueur 2 trahit
    | 1, 0 => 0  -- (T, C)
    | 1, 1 => 1  -- (T, T)
    | _, _ => 0
}

#check prisonersDilemma
──────▶  prisonersDilemma : Game2x2

-- Verification des gains
#eval prisonersDilemma.payoff1 ⟨0, by omega⟩ ⟨0, by omega⟩  -- C,C -> 3
─────▶  3
#eval prisonersDilemma.payoff1 ⟨1, by omega⟩ ⟨0, by omega⟩  -- T,C -> 5
─────▶  5
#eval prisonersDilemma.payoff1 ⟨1, by omega⟩ ⟨1, by omega⟩  -- T,T -> 1
─────▶  1
--% env 11

Raw input:
{"cmd": "-- Dilemme du Prisonnier\n-- Actions : 0 = Cooperer, 1 = Trahir\n-- Gains : (C,C)=(3,3), (C,T)=(0,5), (T,C)=(5,0), (T,T)=(1,1)\n\ndef prisonersDilemma : Game2x2 := {\n  payoff1 := fun i j =>\n    match i.val, j.val with\n    | 0, 0 => 3  -- (C, C)\n    | 0, 1 => 0  -- (C, T)\n    | 1, 0 => 5  -- (T, C)\n    | 1, 1 => 1  -- (T, T)\n    | _, _ => 0\n  payoff2 := fun i j =>\n    match i.val, j.val with\n    | 0, 0 => 3  -- (C, C)\n    | 0, 1 => 5  -- (C, T) - Joueur 2 trahit\n    | 1, 0 => 0  -- (T, C)\n    | 1, 1 => 1  -- (T, T)\n    | _, _ => 0\n}\n\n#check prisonersDilemma\n\n-- Verification des gains\n#eval prisonersDilemma.payoff1 \u27e80, by omega\u27e9 \u27e80, by omega\u27e9  -- C,C -> 3\n#eval prisonersDilemma.payoff1 \u27e81, by omega\u27e9 \u27e80, by omega\u27e9  -- T,C -> 5\n#eval prisonersDilemma.payoff1 \u27e81, by omega\u27e9 \u27e81, by omega\u27e9  -- T,T -> 1", "env": 10}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 22, "column": 0},
   "endPos": {"line": 22, "column": 6},
   "data": "prisonersDilemma : Game2x2"},
  {"severity": "info",
   "pos": {"line": 25, "column": 0},
   "endPos": {"line": 25, "column": 5},
   "data": "3"},
  {"severity": "info",
   "pos": {"line": 26, "column": 0},
   "endPos": {"line": 26, "column": 5},
   "data": "5"},
  {"severity": "info",
   "pos": {"line": 27, "column": 0},
   "endPos": {"line": 27, "column": 5},
   "data": "1"}],
 "env": 11}

### 5.2 Verification que (Trahir, Trahir) est un equilibre de Nash

Prouvons formellement que (T, T) est l'unique equilibre de Nash en strategies pures :

In [13]:
-- Theoreme : (Trahir, Trahir) est un equilibre de Nash du Dilemme du Prisonnier

-- D'abord, definissons les actions
def Cooperer : Fin 2 := ⟨0, by omega⟩
def Trahir : Fin 2 := ⟨1, by omega⟩

-- Theoreme principal : (Trahir, Trahir) est un equilibre de Nash
-- On prouve en utilisant cases sur les valeurs de Fin 2 et omega pour conclure
theorem prisoners_dilemma_nash : isPureNashEquilibrium prisonersDilemma Trahir Trahir := by
  constructor
  -- Joueur 1 : Trahir est meilleure reponse quand J2 trahit
  · intro a1
    simp only [prisonersDilemma, Trahir]
    cases Decidable.em (a1.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a1.val = 1 := by omega
      simp only [this]
      omega
  -- Joueur 2 : Trahir est meilleure reponse quand J1 trahit
  · intro a2
    simp only [prisonersDilemma, Trahir]
    cases Decidable.em (a2.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a2.val = 1 := by omega
      simp only [this]
      omega

#check prisoners_dilemma_nash

-- Theoreme : (Trahir, Trahir) est un equilibre de Nash du Dilemme du Prisonnier

-- D'abord, definissons les actions
def Cooperer : Fin 2 := ⟨0, by omega⟩
def Trahir : Fin 2 := ⟨1, by omega⟩

-- Theoreme principal : (Trahir, Trahir) est un equilibre de Nash
-- On prouve en utilisant cases sur les valeurs de Fin 2 et omega pour conclure
theorem prisoners_dilemma_nash : isPureNashEquilibrium prisonersDilemma Trahir Trahir := by
  constructor
  -- Joueur 1 : Trahir est meilleure reponse quand J2 trahit
  · intro a1
    simp only [prisonersDilemma, Trahir]
    cases Decidable.em (a1.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a1.val = 1 := by omega
      simp only [this]
      omega
  -- Joueur 2 : Trahir est meilleure reponse quand J1 trahit
  · intro a2
    simp only [prisonersDilemma, Trahir]
    cases Decidable.em (a2.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a2.val = 1 := by omega
      simp only [this]
      omega

#check prisoners_dilemma_nash
──────▶  prisoners_dilemma_nash : isPureNashEquilibrium prisonersDilemma Trahir Trahir
--% env 12

Raw input:
{"cmd": "-- Theoreme : (Trahir, Trahir) est un equilibre de Nash du Dilemme du Prisonnier\n\n-- D'abord, definissons les actions\ndef Cooperer : Fin 2 := \u27e80, by omega\u27e9\ndef Trahir : Fin 2 := \u27e81, by omega\u27e9\n\n-- Theoreme principal : (Trahir, Trahir) est un equilibre de Nash\n-- On prouve en utilisant cases sur les valeurs de Fin 2 et omega pour conclure\ntheorem prisoners_dilemma_nash : isPureNashEquilibrium prisonersDilemma Trahir Trahir := by\n  constructor\n  -- Joueur 1 : Trahir est meilleure reponse quand J2 trahit\n  \u00b7 intro a1\n    simp only [prisonersDilemma, Trahir]\n    cases Decidable.em (a1.val = 0) with\n    | inl h =>\n      simp only [h]\n      omega\n    | inr h =>\n      have : a1.val = 1 := by omega\n      simp only [this]\n      omega\n  -- Joueur 2 : Trahir est meilleure reponse quand J1 trahit\n  \u00b7 intro a2\n    simp only [prisonersDilemma, Trahir]\n    cases Decidable.em (a2.val = 0) with\n    | inl h =>\n      simp only [h]\n      omega\n    | inr h =>\n      have : a2.val = 1 := by omega\n      simp only [this]\n      omega\n\n#check prisoners_dilemma_nash", "env": 11}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 34, "column": 0},
   "endPos": {"line": 34, "column": 6},
   "data":
   "prisoners_dilemma_nash : isPureNashEquilibrium prisonersDilemma Trahir Trahir"}],
 "env": 12}

### 5.3 Verification que (C, C) n'est PAS un equilibre

Montrons que (Cooperer, Cooperer) n'est pas un equilibre car chaque joueur a interet a devier :

In [14]:
-- (Cooperer, Cooperer) n'est PAS un equilibre de Nash
-- Car le joueur 1 peut ameliorer en passant a Trahir : 3 < 5

theorem cooperer_not_nash : ¬ isPureNashEquilibrium prisonersDilemma Cooperer Cooperer := by
  intro h
  -- h.1 dit que Cooperer est meilleure reponse pour J1
  -- Donc payoff1(Cooperer, Cooperer) >= payoff1(Trahir, Cooperer)
  -- Mais payoff1(C,C) = 3 et payoff1(T,C) = 5, donc 3 >= 5 est faux
  have h1 := h.1 Trahir
  simp only [Cooperer, Trahir, prisonersDilemma] at h1
  omega

#check cooperer_not_nash

-- (Cooperer, Cooperer) n'est PAS un equilibre de Nash
-- Car le joueur 1 peut ameliorer en passant a Trahir : 3 < 5

theorem cooperer_not_nash : ¬ isPureNashEquilibrium prisonersDilemma Cooperer Cooperer := by
  intro h
  -- h.1 dit que Cooperer est meilleure reponse pour J1
  -- Donc payoff1(Cooperer, Cooperer) >= payoff1(Trahir, Cooperer)
  -- Mais payoff1(C,C) = 3 et payoff1(T,C) = 5, donc 3 >= 5 est faux
  have h1 := h.1 Trahir
  simp only [Cooperer, Trahir, prisonersDilemma] at h1
  omega

#check cooperer_not_nash
──────▶  cooperer_not_nash : ¬isPureNashEquilibrium prisonersDilemma Cooperer Cooperer
--% env 13

Raw input:
{"cmd": "-- (Cooperer, Cooperer) n'est PAS un equilibre de Nash\n-- Car le joueur 1 peut ameliorer en passant a Trahir : 3 < 5\n\ntheorem cooperer_not_nash : \u00ac isPureNashEquilibrium prisonersDilemma Cooperer Cooperer := by\n  intro h\n  -- h.1 dit que Cooperer est meilleure reponse pour J1\n  -- Donc payoff1(Cooperer, Cooperer) >= payoff1(Trahir, Cooperer)\n  -- Mais payoff1(C,C) = 3 et payoff1(T,C) = 5, donc 3 >= 5 est faux\n  have h1 := h.1 Trahir\n  simp only [Cooperer, Trahir, prisonersDilemma] at h1\n  omega\n\n#check cooperer_not_nash", "env": 12}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 13, "column": 0},
   "endPos": {"line": 13, "column": 6},
   "data":
   "cooperer_not_nash : ¬isPureNashEquilibrium prisonersDilemma Cooperer Cooperer"}],
 "env": 13}

### 5.4 Dominance stricte

Dans le Dilemme du Prisonnier, Trahir **domine strictement** Cooperer :

In [15]:
-- Definition de la dominance stricte
def strictlyDominates1 (g : Game2x2) (a a' : Fin 2) : Prop :=
  ∀ a2 : Fin 2, g.payoff1 a a2 > g.payoff1 a' a2

-- Theoreme : Dans le Dilemme du Prisonnier, Trahir domine strictement Cooperer
-- Trahir donne toujours un meilleur gain que Cooperer, peu importe ce que fait J2
theorem trahir_dominates_cooperer : strictlyDominates1 prisonersDilemma Trahir Cooperer := by
  intro a2
  simp only [prisonersDilemma, Trahir, Cooperer]
  cases Decidable.em (a2.val = 0) with
  | inl h =>
    simp only [h]
    omega
  | inr h =>
    have : a2.val = 1 := by omega
    simp only [this]
    omega

#check trahir_dominates_cooperer

-- Corollaire : Une strategie strictement dominante est toujours jouee a l'equilibre
-- (Ceci est une propriete generale, pas specifique au PD)

-- Definition de la dominance stricte
def strictlyDominates1 (g : Game2x2) (a a' : Fin 2) : Prop :=
  ∀ a2 : Fin 2, g.payoff1 a a2 > g.payoff1 a' a2

-- Theoreme : Dans le Dilemme du Prisonnier, Trahir domine strictement Cooperer
-- Trahir donne toujours un meilleur gain que Cooperer, peu importe ce que fait J2
theorem trahir_dominates_cooperer : strictlyDominates1 prisonersDilemma Trahir Cooperer := by
  intro a2
  simp only [prisonersDilemma, Trahir, Cooperer]
  cases Decidable.em (a2.val = 0) with
  | inl h =>
    simp only [h]
    omega
  | inr h =>
    have : a2.val = 1 := by omega
    simp only [this]
    omega

#check trahir_dominates_cooperer
──────▶  trahir_dominates_cooperer : strictlyDominates1 prisonersDilemma Trahir Cooperer

-- Corollaire : Une strategie strictement dominante est toujours jouee a l'equilibre
-- (Ceci est une propriete generale, pas specifique au PD)
--% env 14

Raw input:
{"cmd": "-- Definition de la dominance stricte\ndef strictlyDominates1 (g : Game2x2) (a a' : Fin 2) : Prop :=\n  \u2200 a2 : Fin 2, g.payoff1 a a2 > g.payoff1 a' a2\n\n-- Theoreme : Dans le Dilemme du Prisonnier, Trahir domine strictement Cooperer\n-- Trahir donne toujours un meilleur gain que Cooperer, peu importe ce que fait J2\ntheorem trahir_dominates_cooperer : strictlyDominates1 prisonersDilemma Trahir Cooperer := by\n  intro a2\n  simp only [prisonersDilemma, Trahir, Cooperer]\n  cases Decidable.em (a2.val = 0) with\n  | inl h =>\n    simp only [h]\n    omega\n  | inr h =>\n    have : a2.val = 1 := by omega\n    simp only [this]\n    omega\n\n#check trahir_dominates_cooperer\n\n-- Corollaire : Une strategie strictement dominante est toujours jouee a l'equilibre\n-- (Ceci est une propriete generale, pas specifique au PD)", "env": 13}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 19, "column": 0},
   "endPos": {"line": 19, "column": 6},
   "data":
   "trahir_dominates_cooperer : strictlyDominates1 prisonersDilemma Trahir Cooperer"}],
 "env": 14}

<a id="6-exercices"></a>
## 6. Exercices

### Exercice 1 : Jeu de la Poule Mouillee (Chicken)

Definir le jeu Chicken et prouver qu'il a deux equilibres de Nash en strategies pures.

|  | Ceder (0) | Foncer (1) |
|--|-----------|------------|
| **Ceder (0)** | (3, 3) | (2, 4) |
| **Foncer (1)** | (4, 2) | (0, 0) |

**Questions** :
1. Definir `chickenGame : Game2x2`
2. Prouver que (Foncer, Ceder) est un equilibre de Nash
3. Prouver que (Ceder, Foncer) est un equilibre de Nash

### Exercice 2 : Matching Pennies

Definir le jeu Matching Pennies (jeu a somme nulle) et montrer qu'il n'a pas d'equilibre de Nash en strategies pures.

|  | Pile (0) | Face (1) |
|--|----------|----------|
| **Pile (0)** | (1, -1) | (-1, 1) |
| **Face (1)** | (-1, 1) | (1, -1) |

**Questions** :
1. Definir `matchingPennies : Game2x2`
2. Prouver qu'aucune des 4 paires d'actions pures n'est un equilibre

### Exercice 3 : Chasse au Cerf (Stag Hunt)

Definir le jeu Stag Hunt et prouver qu'il a deux equilibres de Nash purs.

|  | Cerf (0) | Lievre (1) |
|--|----------|------------|
| **Cerf (0)** | (4, 4) | (0, 3) |
| **Lievre (1)** | (3, 0) | (2, 2) |

**Questions** :
1. Definir `stagHunt : Game2x2`
2. Prouver que (Cerf, Cerf) est un equilibre (Pareto-optimal)
3. Prouver que (Lievre, Lievre) est aussi un equilibre (risk-dominant)

<a id="7-solutions"></a>
## 7. Solutions

### Solution Exercice 1 : Chicken

In [16]:
-- Solution Exercice 1 : Jeu de la Poule Mouillee (Chicken)

def chickenGame : Game2x2 := {
  payoff1 := fun i j =>
    match i.val, j.val with
    | 0, 0 => 3  -- (Ceder, Ceder)
    | 0, 1 => 2  -- (Ceder, Foncer)
    | 1, 0 => 4  -- (Foncer, Ceder)
    | 1, 1 => 0  -- (Foncer, Foncer) - crash!
    | _, _ => 0
  payoff2 := fun i j =>
    match i.val, j.val with
    | 0, 0 => 3
    | 0, 1 => 4  -- J2 fonce, J1 cede
    | 1, 0 => 2  -- J1 fonce, J2 cede
    | 1, 1 => 0
    | _, _ => 0
}

-- Actions
def Ceder : Fin 2 := ⟨0, by omega⟩
def Foncer : Fin 2 := ⟨1, by omega⟩

-- Equilibre 1 : (Foncer, Ceder)
-- J1 joue Foncer, J2 joue Ceder : gains (4, 2)
-- J1 ne peut pas ameliorer : 4 >= 3 (si Ceder)
-- J2 ne peut pas ameliorer : 2 >= 0 (si Foncer)
theorem chicken_nash1 : isPureNashEquilibrium chickenGame Foncer Ceder := by
  constructor
  · intro a1
    simp only [chickenGame, Foncer, Ceder]
    cases Decidable.em (a1.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a1.val = 1 := by omega
      simp only [this]
      omega
  · intro a2
    simp only [chickenGame, Foncer, Ceder]
    cases Decidable.em (a2.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a2.val = 1 := by omega
      simp only [this]
      omega

-- Equilibre 2 : (Ceder, Foncer)
theorem chicken_nash2 : isPureNashEquilibrium chickenGame Ceder Foncer := by
  constructor
  · intro a1
    simp only [chickenGame, Foncer, Ceder]
    cases Decidable.em (a1.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a1.val = 1 := by omega
      simp only [this]
      omega
  · intro a2
    simp only [chickenGame, Foncer, Ceder]
    cases Decidable.em (a2.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a2.val = 1 := by omega
      simp only [this]
      omega

#check chicken_nash1
#check chicken_nash2

-- Solution Exercice 1 : Jeu de la Poule Mouillee (Chicken)

def chickenGame : Game2x2 := {
  payoff1 := fun i j =>
    match i.val, j.val with
    | 0, 0 => 3  -- (Ceder, Ceder)
    | 0, 1 => 2  -- (Ceder, Foncer)
    | 1, 0 => 4  -- (Foncer, Ceder)
    | 1, 1 => 0  -- (Foncer, Foncer) - crash!
    | _, _ => 0
  payoff2 := fun i j =>
    match i.val, j.val with
    | 0, 0 => 3
    | 0, 1 => 4  -- J2 fonce, J1 cede
    | 1, 0 => 2  -- J1 fonce, J2 cede
    | 1, 1 => 0
    | _, _ => 0
}

-- Actions
def Ceder : Fin 2 := ⟨0, by omega⟩
def Foncer : Fin 2 := ⟨1, by omega⟩

-- Equilibre 1 : (Foncer, Ceder)
-- J1 joue Foncer, J2 joue Ceder : gains (4, 2)
-- J1 ne peut pas ameliorer : 4 >= 3 (si Ceder)
-- J2 ne peut pas ameliorer : 2 >= 0 (si Foncer)
theorem chicken_nash1 : isPureNashEquilibrium chickenGame Foncer Ceder := by
  constructor
  · intro a1
    simp only [chickenGame, Foncer, Ceder]
    cases Decidable.em (a1.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a1.val = 1 := by omega
      simp only [this]
      omega
  · intro a2
    simp only [chickenGame, Foncer, Ceder]
    cases Decidable.em (a2.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a2.val = 1 := by omega
      simp only [this]
      omega

-- Equilibre 2 : (Ceder, Foncer)
theorem chicken_nash2 : isPureNashEquilibrium chickenGame Ceder Foncer := by
  constructor
  · intro a1
    simp only [chickenGame, Foncer, Ceder]
    cases Decidable.em (a1.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a1.val = 1 := by omega
      simp only [this]
      omega
  · intro a2
    simp only [chickenGame, Foncer, Ceder]
    cases Decidable.em (a2.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a2.val = 1 := by omega
      simp only [this]
      omega

#check chicken_nash1
──────▶  chicken_nash1 : isPureNashEquilibrium chickenGame Foncer Ceder
#check chicken_nash2
──────▶  chicken_nash2 : isPureNashEquilibrium chickenGame Ceder Foncer
--% env 15

Raw input:
{"cmd": "-- Solution Exercice 1 : Jeu de la Poule Mouillee (Chicken)\n\ndef chickenGame : Game2x2 := {\n  payoff1 := fun i j =>\n    match i.val, j.val with\n    | 0, 0 => 3  -- (Ceder, Ceder)\n    | 0, 1 => 2  -- (Ceder, Foncer)\n    | 1, 0 => 4  -- (Foncer, Ceder)\n    | 1, 1 => 0  -- (Foncer, Foncer) - crash!\n    | _, _ => 0\n  payoff2 := fun i j =>\n    match i.val, j.val with\n    | 0, 0 => 3\n    | 0, 1 => 4  -- J2 fonce, J1 cede\n    | 1, 0 => 2  -- J1 fonce, J2 cede\n    | 1, 1 => 0\n    | _, _ => 0\n}\n\n-- Actions\ndef Ceder : Fin 2 := \u27e80, by omega\u27e9\ndef Foncer : Fin 2 := \u27e81, by omega\u27e9\n\n-- Equilibre 1 : (Foncer, Ceder)\n-- J1 joue Foncer, J2 joue Ceder : gains (4, 2)\n-- J1 ne peut pas ameliorer : 4 >= 3 (si Ceder)\n-- J2 ne peut pas ameliorer : 2 >= 0 (si Foncer)\ntheorem chicken_nash1 : isPureNashEquilibrium chickenGame Foncer Ceder := by\n  constructor\n  \u00b7 intro a1\n    simp only [chickenGame, Foncer, Ceder]\n    cases Decidable.em (a1.val = 0) with\n    | inl h =>\n      simp only [h]\n      omega\n    | inr h =>\n      have : a1.val = 1 := by omega\n      simp only [this]\n      omega\n  \u00b7 intro a2\n    simp only [chickenGame, Foncer, Ceder]\n    cases Decidable.em (a2.val = 0) with\n    | inl h =>\n      simp only [h]\n      omega\n    | inr h =>\n      have : a2.val = 1 := by omega\n      simp only [this]\n      omega\n\n-- Equilibre 2 : (Ceder, Foncer)\ntheorem chicken_nash2 : isPureNashEquilibrium chickenGame Ceder Foncer := by\n  constructor\n  \u00b7 intro a1\n    simp only [chickenGame, Foncer, Ceder]\n    cases Decidable.em (a1.val = 0) with\n    | inl h =>\n      simp only [h]\n      omega\n    | inr h =>\n      have : a1.val = 1 := by omega\n      simp only [this]\n      omega\n  \u00b7 intro a2\n    simp only [chickenGame, Foncer, Ceder]\n    cases Decidable.em (a2.val = 0) with\n    | inl h =>\n      simp onl

### Solution Exercice 2 : Matching Pennies

In [17]:
-- Solution Exercice 2 : Matching Pennies

def matchingPennies : Game2x2 := {
  payoff1 := fun i j =>
    match i.val, j.val with
    | 0, 0 => 1   -- (Pile, Pile) - J1 gagne
    | 0, 1 => -1  -- (Pile, Face)
    | 1, 0 => -1  -- (Face, Pile)
    | 1, 1 => 1   -- (Face, Face) - J1 gagne
    | _, _ => 0
  payoff2 := fun i j =>
    match i.val, j.val with
    | 0, 0 => -1  -- J2 veut des resultats differents
    | 0, 1 => 1
    | 1, 0 => 1
    | 1, 1 => -1
    | _, _ => 0
}

def Pile : Fin 2 := ⟨0, by omega⟩
def Face : Fin 2 := ⟨1, by omega⟩

-- Aucune des 4 paires n'est un equilibre
-- (Pile, Pile) : J2 prefere devier vers Face (-1 -> 1)
theorem matching_pennies_no_pure_nash_00 : 
    ¬ isPureNashEquilibrium matchingPennies Pile Pile := by
  intro h
  have := h.2 Face  -- J2 prefere Face quand J1 joue Pile: payoff2(Pile, Face) = 1 > -1 = payoff2(Pile, Pile)
  simp only [Pile, Face, matchingPennies] at this
  omega

-- (Pile, Face) : J1 prefere devier vers Face (car Face, Face gagne pour J1)
theorem matching_pennies_no_pure_nash_01 :
    ¬ isPureNashEquilibrium matchingPennies Pile Face := by
  intro h
  have := h.1 Face  -- J1 prefere Face quand J2 joue Face
  simp only [Pile, Face, matchingPennies] at this
  omega

-- (Face, Pile) : J1 prefere devier vers Pile
theorem matching_pennies_no_pure_nash_10 :
    ¬ isPureNashEquilibrium matchingPennies Face Pile := by
  intro h
  have := h.1 Pile  -- J1 prefere Pile quand J2 joue Pile
  simp only [Pile, Face, matchingPennies] at this
  omega

-- (Face, Face) : J2 prefere devier vers Pile
theorem matching_pennies_no_pure_nash_11 :
    ¬ isPureNashEquilibrium matchingPennies Face Face := by
  intro h
  have := h.2 Pile  -- J2 prefere Pile quand J1 joue Face
  simp only [Pile, Face, matchingPennies] at this
  omega

#check matching_pennies_no_pure_nash_00
#check matching_pennies_no_pure_nash_01
#check matching_pennies_no_pure_nash_10
#check matching_pennies_no_pure_nash_11

-- Solution Exercice 2 : Matching Pennies

def matchingPennies : Game2x2 := {
  payoff1 := fun i j =>
    match i.val, j.val with
    | 0, 0 => 1   -- (Pile, Pile) - J1 gagne
    | 0, 1 => -1  -- (Pile, Face)
    | 1, 0 => -1  -- (Face, Pile)
    | 1, 1 => 1   -- (Face, Face) - J1 gagne
    | _, _ => 0
  payoff2 := fun i j =>
    match i.val, j.val with
    | 0, 0 => -1  -- J2 veut des resultats differents
    | 0, 1 => 1
    | 1, 0 => 1
    | 1, 1 => -1
    | _, _ => 0
}

def Pile : Fin 2 := ⟨0, by omega⟩
def Face : Fin 2 := ⟨1, by omega⟩

-- Aucune des 4 paires n'est un equilibre
-- (Pile, Pile) : J2 prefere devier vers Face (-1 -> 1)
theorem matching_pennies_no_pure_nash_00 : 
    ¬ isPureNashEquilibrium matchingPennies Pile Pile := by
  intro h
  have := h.2 Face  -- J2 prefere Face quand J1 joue Pile: payoff2(Pile, Face) = 1 > -1 = payoff2(Pile, Pile)
  simp only [Pile, Face, matchingPennies] at this
  omega

-- (Pile, Face) : J1 prefere devier vers Face (car Face, Face gagne pour J1)
theorem matching_pennies_no_pure_nash_01 :
    ¬ isPureNashEquilibrium matchingPennies Pile Face := by
  intro h
  have := h.1 Face  -- J1 prefere Face quand J2 joue Face
  simp only [Pile, Face, matchingPennies] at this
  omega

-- (Face, Pile) : J1 prefere devier vers Pile
theorem matching_pennies_no_pure_nash_10 :
    ¬ isPureNashEquilibrium matchingPennies Face Pile := by
  intro h
  have := h.1 Pile  -- J1 prefere Pile quand J2 joue Pile
  simp only [Pile, Face, matchingPennies] at this
  omega

-- (Face, Face) : J2 prefere devier vers Pile
theorem matching_pennies_no_pure_nash_11 :
    ¬ isPureNashEquilibrium matchingPennies Face Face := by
  intro h
  have := h.2 Pile  -- J2 prefere Pile quand J1 joue Face
  simp only [Pile, Face, matchingPennies] at this
  omega

#check matching_pennies_no_pure_nash_00
──────▶  matching_pennies_no_pure_nash_00 : ¬isPureNashEquilibrium matchingPennies Pile Pile
#check matching_pennies_no_pure_nash_01
──────▶  matching_pennies_no_pure_nash_01 : ¬isPureNashEquilibrium matchingPennies Pile Face
#check matching_pennies_no_pure_nash_10
──────▶  matching_pennies_no_pure_nash_10 : ¬isPureNashEquilibrium matchingPennies Face Pile
#check matching_pennies_no_pure_nash_11
──────▶  matching_pennies_no_pure_nash_11 : ¬isPureNashEquilibrium matchingPennies Face Face
--% env 16

Raw input:
{"cmd": "-- Solution Exercice 2 : Matching Pennies\n\ndef matchingPennies : Game2x2 := {\n  payoff1 := fun i j =>\n    match i.val, j.val with\n    | 0, 0 => 1   -- (Pile, Pile) - J1 gagne\n    | 0, 1 => -1  -- (Pile, Face)\n    | 1, 0 => -1  -- (Face, Pile)\n    | 1, 1 => 1   -- (Face, Face) - J1 gagne\n    | _, _ => 0\n  payoff2 := fun i j =>\n    match i.val, j.val with\n    | 0, 0 => -1  -- J2 veut des resultats differents\n    | 0, 1 => 1\n    | 1, 0 => 1\n    | 1, 1 => -1\n    | _, _ => 0\n}\n\ndef Pile : Fin 2 := \u27e80, by omega\u27e9\ndef Face : Fin 2 := \u27e81, by omega\u27e9\n\n-- Aucune des 4 paires n'est un equilibre\n-- (Pile, Pile) : J2 prefere devier vers Face (-1 -> 1)\ntheorem matching_pennies_no_pure_nash_00 : \n    \u00ac isPureNashEquilibrium matchingPennies Pile Pile := by\n  intro h\n  have := h.2 Face  -- J2 prefere Face quand J1 joue Pile: payoff2(Pile, Face) = 1 > -1 = payoff2(Pile, Pile)\n  simp only [Pile, Face, matchingPennies] at this\n  omega\n\n-- (Pile, Face) : J1 prefere devier vers Face (car Face, Face gagne pour J1)\ntheorem matching_pennies_no_pure_nash_01 :\n    \u00ac isPureNashEquilibrium matchingPennies Pile Face := by\n  intro h\n  have := h.1 Face  -- J1 prefere Face quand J2 joue Face\n  simp only [Pile, Face, matchingPennies] at this\n  omega\n\n-- (Face, Pile) : J1 prefere devier vers Pile\ntheorem matching_pennies_no_pure_nash_10 :\n    \u00ac isPureNashEquilibrium matchingPennies Face Pile := by\n  intro h\n  have := h.1 Pile  -- J1 prefere Pile quand J2 joue Pile\n  simp only [Pile, Face, matchingPennies] at this\n  omega\n\n-- (Face, Face) : J2 prefere devier vers Pile\ntheorem m

### Solution Exercice 3 : Stag Hunt

In [18]:
-- Solution Exercice 3 : Chasse au Cerf (Stag Hunt)

def stagHunt : Game2x2 := {
  payoff1 := fun i j =>
    match i.val, j.val with
    | 0, 0 => 4  -- (Cerf, Cerf) - succes!
    | 0, 1 => 0  -- (Cerf, Lievre) - J1 echoue seul
    | 1, 0 => 3  -- (Lievre, Cerf)
    | 1, 1 => 2  -- (Lievre, Lievre)
    | _, _ => 0
  payoff2 := fun i j =>
    match i.val, j.val with
    | 0, 0 => 4
    | 0, 1 => 3
    | 1, 0 => 0
    | 1, 1 => 2
    | _, _ => 0
}

def Cerf : Fin 2 := ⟨0, by omega⟩
def Lievre : Fin 2 := ⟨1, by omega⟩

-- Equilibre Pareto-optimal : (Cerf, Cerf) - gains (4, 4)
theorem stag_hunt_nash_cerf : isPureNashEquilibrium stagHunt Cerf Cerf := by
  constructor
  · intro a1
    simp only [stagHunt, Cerf]
    cases Decidable.em (a1.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a1.val = 1 := by omega
      simp only [this]
      omega
  · intro a2
    simp only [stagHunt, Cerf]
    cases Decidable.em (a2.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a2.val = 1 := by omega
      simp only [this]
      omega

-- Equilibre risk-dominant : (Lievre, Lievre) - gains (2, 2)
theorem stag_hunt_nash_lievre : isPureNashEquilibrium stagHunt Lievre Lievre := by
  constructor
  · intro a1
    simp only [stagHunt, Lievre]
    cases Decidable.em (a1.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a1.val = 1 := by omega
      simp only [this]
      omega
  · intro a2
    simp only [stagHunt, Lievre]
    cases Decidable.em (a2.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a2.val = 1 := by omega
      simp only [this]
      omega

#check stag_hunt_nash_cerf
#check stag_hunt_nash_lievre

-- Solution Exercice 3 : Chasse au Cerf (Stag Hunt)

def stagHunt : Game2x2 := {
  payoff1 := fun i j =>
    match i.val, j.val with
    | 0, 0 => 4  -- (Cerf, Cerf) - succes!
    | 0, 1 => 0  -- (Cerf, Lievre) - J1 echoue seul
    | 1, 0 => 3  -- (Lievre, Cerf)
    | 1, 1 => 2  -- (Lievre, Lievre)
    | _, _ => 0
  payoff2 := fun i j =>
    match i.val, j.val with
    | 0, 0 => 4
    | 0, 1 => 3
    | 1, 0 => 0
    | 1, 1 => 2
    | _, _ => 0
}

def Cerf : Fin 2 := ⟨0, by omega⟩
def Lievre : Fin 2 := ⟨1, by omega⟩

-- Equilibre Pareto-optimal : (Cerf, Cerf) - gains (4, 4)
theorem stag_hunt_nash_cerf : isPureNashEquilibrium stagHunt Cerf Cerf := by
  constructor
  · intro a1
    simp only [stagHunt, Cerf]
    cases Decidable.em (a1.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a1.val = 1 := by omega
      simp only [this]
      omega
  · intro a2
    simp only [stagHunt, Cerf]
    cases Decidable.em (a2.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a2.val = 1 := by omega
      simp only [this]
      omega

-- Equilibre risk-dominant : (Lievre, Lievre) - gains (2, 2)
theorem stag_hunt_nash_lievre : isPureNashEquilibrium stagHunt Lievre Lievre := by
  constructor
  · intro a1
    simp only [stagHunt, Lievre]
    cases Decidable.em (a1.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a1.val = 1 := by omega
      simp only [this]
      omega
  · intro a2
    simp only [stagHunt, Lievre]
    cases Decidable.em (a2.val = 0) with
    | inl h =>
      simp only [h]
      omega
    | inr h =>
      have : a2.val = 1 := by omega
      simp only [this]
      omega

#check stag_hunt_nash_cerf
──────▶  stag_hunt_nash_cerf : isPureNashEquilibrium stagHunt Cerf Cerf
#check stag_hunt_nash_lievre
──────▶  stag_hunt_nash_lievre : isPureNashEquilibrium stagHunt Lievre Lievre
--% env 17

Raw input:
{"cmd": "-- Solution Exercice 3 : Chasse au Cerf (Stag Hunt)\n\ndef stagHunt : Game2x2 := {\n  payoff1 := fun i j =>\n    match i.val, j.val with\n    | 0, 0 => 4  -- (Cerf, Cerf) - succes!\n    | 0, 1 => 0  -- (Cerf, Lievre) - J1 echoue seul\n    | 1, 0 => 3  -- (Lievre, Cerf)\n    | 1, 1 => 2  -- (Lievre, Lievre)\n    | _, _ => 0\n  payoff2 := fun i j =>\n    match i.val, j.val with\n    | 0, 0 => 4\n    | 0, 1 => 3\n    | 1, 0 => 0\n    | 1, 1 => 2\n    | _, _ => 0\n}\n\ndef Cerf : Fin 2 := \u27e80, by omega\u27e9\ndef Lievre : Fin 2 := \u27e81, by omega\u27e9\n\n-- Equilibre Pareto-optimal : (Cerf, Cerf) - gains (4, 4)\ntheorem stag_hunt_nash_cerf : isPureNashEquilibrium stagHunt Cerf Cerf := by\n  constructor\n  \u00b7 intro a1\n    simp only [stagHunt, Cerf]\n    cases Decidable.em (a1.val = 0) with\n    | inl h =>\n      simp only [h]\n      omega\n    | inr h =>\n      have : a1.val = 1 := by omega\n      simp only [this]\n      omega\n  \u00b7 intro a2\n    simp only [stagHunt, Cerf]\n    cases Decidable.em (a2.val = 0) with\n    | inl h =>\n      simp only [h]\n      omega\n    | inr h =>\n      have : a2.val = 1 := by omega\n      simp only [this]\n      omega\n\n-- Equilibre risk-dominant : (Lievre, Lievre) - gains (2, 2)\ntheorem stag_hunt_nash_lievre : isPureNashEquilibrium stagHunt Lievre Lievre := by\n  constructor\n  \u00b7 intro a1\n    simp only [stagHunt, Lievre]\n    cases Decidable.em (a1.val = 0) with\n    | inl h =>\n      simp only [h]\n      omega\n    | inr h =>\n      have : a1.val = 1 := by omega\n      simp only [this]\n      omega\n  \u00b7 intro a2\n    simp only [stagHunt, Lievre]\n    cases Decidable.em (a2.val = 0) with\n    | inl h =>\n      simp only [h]\n      omega\n    | inr h =>\n      have : a2.val = 1 := by omega\n      simp only [this]\n      omega\n\n#check stag_hunt_nash_cerf\n#check stag_hunt_nash_lievre", "env": 16}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 71, "column": 0},
   "endPos": {"line": 71, "column": 6},
   "data": "stag_hunt_nash_cerf : isPur

<a id="8-resume"></a>
## 8. Resume

### Concepts formalises

| Concept | Definition Lean | Description |
|---------|-----------------|-------------|
| **Jeu en forme normale** | `NormalFormGame`, `FiniteGame`, `Game2x2` | Joueurs, actions, gains |
| **Strategie pure** | `Fin n` | Une action deterministe |
| **Strategie mixte** | `Simplex n` | Distribution sur les actions |
| **Gain espere** | `expectedPayoff1`, `expectedPayoff2` | Esperance sous strategies mixtes |
| **Equilibre de Nash** | `isNashEquilibrium`, `isPureNashEquilibrium` | Aucune deviation profitable |
| **Dominance stricte** | `strictlyDominates1` | Une action toujours meilleure |

### Theoremes prouves

| Theoreme | Enonce |
|----------|--------|
| `prisoners_dilemma_nash` | (T, T) est equilibre du PD |
| `cooperer_not_nash` | (C, C) n'est pas equilibre du PD |
| `trahir_dominates_cooperer` | Trahir domine strictement Cooperer |
| `chicken_nash1`, `chicken_nash2` | Chicken a 2 equilibres purs |
| `matching_pennies_no_pure_nash_*` | MP n'a pas d'equilibre pur |
| `stag_hunt_nash_cerf`, `stag_hunt_nash_lievre` | Stag Hunt a 2 equilibres purs |

### Prochaines etapes

Dans le notebook suivant (**GameTheory-4b-Lean-NashExistence**), nous etudierons :
- Le theoreme de Brouwer (point fixe)
- L'existence d'un equilibre de Nash dans tout jeu fini
- Lecture guidee du repo `math-xmum/Brouwer`

---

**Navigation** : [← GameTheory-17-MultiAgent-RL](GameTheory-17-MultiAgent-RL.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [GameTheory-4b-Lean-NashExistence →](GameTheory-4b-Lean-NashExistence.ipynb)